yOLO

In [17]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
yolo_model = YOLO("yolo11n.pt")

In [19]:
results = yolo_model("images/ss.jpg")  # Predict on an image
results[0].show()  # Display results


image 1/1 c:\Users\djadh\Downloads\Analyze Gpt\images\ss.jpg: 640x544 1 person, 119.9ms
Speed: 6.5ms preprocess, 119.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 544)


YOLO E

In [14]:
from ultralytics import YOLOE

# Initialize a YOLOE model
model = YOLOE("yoloe-v8s-seg.pt")  # or select yoloe-11s/m-seg.pt for different sizes

# Set text prompt
names = ["paino"]
model.set_classes(names, model.get_text_pe(names))

# Execute prediction for specified categories on an image
results = model.predict("images/ss.jpg")

# Show results
results[0].show()


image 1/1 c:\Users\djadh\Downloads\Analyze Gpt\images\ss.jpg: 640x544 5 painos, 294.5ms
Speed: 2.9ms preprocess, 294.5ms inference, 8.3ms postprocess per image at shape (1, 3, 640, 544)


In [15]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'paino'}
 obb: None
 orig_img: array([[[219, 219, 219],
         [219, 219, 219],
         [219, 219, 219],
         ...,
         [227, 227, 227],
         [227, 227, 227],
         [227, 227, 227]],
 
        [[219, 219, 219],
         [219, 219, 219],
         [219, 219, 219],
         ...,
         [228, 228, 228],
         [228, 228, 228],
         [228, 228, 228]],
 
        [[219, 219, 219],
         [219, 219, 219],
         [219, 219, 219],
         ...,
         [229, 229, 229],
         [229, 229, 229],
         [229, 229, 229]],
 
        ...,
 
        [[ 40,  40,  40],
         [ 41,  41,  41],
         [ 32,  32,  32],
         ...,
         [115, 115, 115],
         [110, 110, 110],
         [113, 113, 113]],
 
        [[ 82,  82,  82],
         [ 88,  88,  88],
         [ 60,  60,  60]

In [20]:
model = YOLOE("yoloe-v8s-seg.pt") 
results = model.predict("images/ss.jpg")
results[0].show()


image 1/1 c:\Users\djadh\Downloads\Analyze Gpt\images\ss.jpg: 640x544 (no detections), 280.1ms
Speed: 3.0ms preprocess, 280.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)


Agents

In [3]:
from autogen_core.models import UserMessage
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent


# Assuming your Ollama server is running locally on port 11434.
ollama_model_client = OllamaChatCompletionClient(model="llama3.2")



gemini model setup

In [43]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from config.constants import GEMINI_MODEL
from dotenv import load_dotenv
import os
load_dotenv()


gemini_model_client = OpenAIChatCompletionClient(
model="gemini-1.5-flash-8b",
api_key=os.getenv('GOOGLE_API_KEY'),
)

TOOLS

In [44]:
from ultralytics import YOLO

yolo_model = YOLO("yolo11n.pt")

def yolo_tool(image_path: str):
    
    
    """
    Runs YOLO object detection, segmentation, and pose estimation on the given image.

    Args:
        image_path (str): Path to the input image.

    Returns:
        list: A list of dictionaries containing detected objects, confidence scores, and bounding boxes.
    
    """
    
    results = yolo_model(image_path)
    results[0].show()
    classes = list({results[0].names[int(box.cls)] for box in results[0].boxes})
    detections = [{
        "class_id": int(box.cls),
        "class_name": results[0].names[int(box.cls)],
        "confidence": round(float(box.conf), 2)
    } for box in results[0].boxes]
    return {"detected_classes": classes, "detailed_detections": detections}

AGENTS

In [53]:
from autogen_agentchat.agents import AssistantAgent


yolo_agent = AssistantAgent(
name = "Object_Detection_agent",
model_client=gemini_model_client,
tools = [yolo_tool],
description="An agent that is an expert in Object detection , segmentation , pose estimation",
system_message="""
You are an AI agent specialized in computer vision tasks, with advanced capabilities in:  
- Object detection  
- Instance segmentation  
- Pose estimation  

You will be provided with an image as input with path
you should give that path to your yolo_tool  
Your task is to use the YOLO Tool to:  
1. Detect all objects present in the image.  
2. Perform segmentation for each detected object.  
3. Conduct pose estimation where applicable.  

After running the YOLO Tool:  
- Summarize the results clearly in natural language, highlighting key 
findings such as object types, counts, and any notable poses or segmentation details.  
- Conclude your response by saying **"TERMINATE"**.  

**Important Rule:**  
Only use the YOLO Tool when the query is directly related to object detection, segmentation, or pose estimation.  
If the query is unrelated, provide the most relevant response instead, and still end with **"TERMINATE"**.
"""
)

TEAMS with termination conditions

In [46]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

team = RoundRobinGroupChat(
    participants=[yolo_agent],
    max_turns=4,
    termination_condition= TextMentionTermination("TERMINATE")
)

Queries

Multimodal Message

In [54]:
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image
from PIL import Image as PILImage
from autogen_agentchat.ui import Console

# Load image from file
pil_image = PILImage.open("images/oo.jpeg")

# Create AutoGen-compatible Image object
img = Image(pil_image)

# Create a multi-modal message
multi_modal_message = MultiModalMessage(
    content=["""Can you describe something about the images and also
             the exact number of people in this image at path 'images/oo.jpeg'""", img],
    source="user" 
)

res = await Console(team.run_stream(task = multi_modal_message))
res.messages[-1].content

---------- MultiModalMessage (user) ----------
Can you describe something about the images and also
             the exact number of people in this image at path 'images/oo.jpeg'
<image>



---------- ToolCallRequestEvent (Object_Detection_agent) ----------
[FunctionCall(id='5', arguments='{"image_path": "images/oo.jpeg"}', name='yolo_tool')]
image 1/1 c:\Users\djadh\Downloads\Analyze Gpt\images\oo.jpeg: 512x640 14 persons, 1 tie, 87.6ms
Speed: 2.0ms preprocess, 87.6ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)
---------- ToolCallExecutionEvent (Object_Detection_agent) ----------
[FunctionExecutionResult(content="{'detected_classes': ['person', 'tie'], 'detailed_detections': [{'class_id': 0, 'class_name': 'person', 'confidence': 0.86}, {'class_id': 0, 'class_name': 'person', 'confidence': 0.81}, {'class_id': 0, 'class_name': 'person', 'confidence': 0.8}, {'class_id': 0, 'class_name': 'person', 'confidence': 0.8}, {'class_id': 0, 'class_name': 'person', 'confidence': 0.72}, {'class_id': 0, 'class_name': 'person', 'confidence': 0.66}, {'class_id': 0, 'class_name': 'person', 'confidence': 0.64}, {'class_id': 0, 'class_name': 'person', 'confidence': 0.6

"The image at path 'images/oo.jpeg' contains multiple people, as well as one person wearing a tie. The exact number of people in the image is 13. These individuals have varying degrees of confidence in their detection, ranging from 0.32 to 0.86.\n\nIt's worth noting that the image likely contains additional details beyond just the detected people and ties, such as clothing, accessories, or background information.\n\n\nTERMINATE"